In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

In [3]:
#from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet, classification_model_vgg
from ordinaloss.utils.utils import create_ordinal_cost_matrix


from ordinaloss.engine.model_engine import OrdinalEngine
from ordinaloss.utils.datasets import data_load

from ordinaloss.utils.loss_utils import SinimLoss, GirlsLoss, CSCELoss


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch
import mlflow

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

loaded ordinaloss.utils.pretrained_models
loaded ordinaloss.utils.datasets
loaded ordinaloss.utils.loss_utils


'cpu'

In [4]:
# train_transform = create_transform_pipeline(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3) #These are hyperparameters. 
# test_transform = create_transform_pipeline() #Without color jitter for test pipeline

# train_dataset = ImageFolder("../datasets/kneeKL224/train/", transform=train_transform)
# val_dataset = ImageFolder("../datasets/kneeKL224/val/", transform = test_transform)
# test_dataset = ImageFolder("../datasets/kneeKL224/test/", transform = test_transform)
# auto_test_dataset = ImageFolder("../datasets/kneeKL224/auto_test/", transform = test_transform)

# train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
# test_loader  = DataLoader(test_dataset,  batch_size = 128, shuffle=False)

loaders, _, _ = data_load("../datasets/kneeKL224/", 16)
train_loader = loaders["train"]
test_loader = loaders["test"]

In [5]:
#model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
model = classification_model_vgg("vgg16", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
# model = classification_model_densenet("densenet121", num_classes = 5) #Could also be densenet169, densenet201

In [6]:
create_ordinal_cost_matrix(5)

tensor([[20.,  1.,  2.,  3.,  4.],
        [ 1., 20.,  1.,  2.,  3.],
        [ 2.,  1., 20.,  1.,  2.],
        [ 3.,  2.,  1., 20.,  1.],
        [ 4.,  3.,  2.,  1., 20.]])

In [40]:
from ordinaloss.utils.loss_utils import GirlsLoss, SinimLoss
model = classification_model_vgg("vgg16", num_classes = 5)

def my_lambda_scheduler(epoch):
    return (0.9 ** (epoch // 5))


# engine = OrdinalEngine(model, 
#                        loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


# engine = OrdinalEngine(model, 
#                        loss_fn = SinimLoss(), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=Adam, lr = 1.0e-3, weight_decay=5.0e-2)                       

In [96]:
create_ordinal_cost_matrix(5, cost_distance=1, diagonal_value=3)

tensor([[3., 2., 3., 4., 5.],
        [2., 3., 2., 3., 4.],
        [3., 2., 3., 2., 3.],
        [4., 3., 2., 3., 2.],
        [5., 4., 3., 2., 3.]])

In [183]:
from itertools import product

param_grid = {
            "lr" : [1e-3, 5e-4, 5e-5],
            "batch_size" : [16,32],
            "cb_matrix_diag_value" : [1,5,10],
            "cost_distance" : [1, 2, 3],
            }

experiments_configuration = pd.DataFrame(list(product(*param_grid.values())), columns = param_grid.keys())
len(experiments_configuration)

54

In [182]:
model = classification_model_vgg("vgg16", num_classes = 5)

engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(create_ordinal_cost_matrix(size = 5, cost_distance=2, diagonal_value=5)), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=Adam, lr = 1.0e-3, weight_decay=5.0e-2)

loaded ordinaloss.engine.model_engine


In [209]:
for _, row in experiments_configuration.iterrows():
    config = row.to_dict()

    model = classification_model_vgg("vgg16", num_classes = 5)

    loaders, _, _ = data_load("../datasets/kneeKL224/", int(config["batch_size"]))
    train_loader = loaders["train"]
    test_loader = loaders["test"]

    cb_matrix = create_ordinal_cost_matrix(size = 5, cost_distance=config["cost_distance"], diagonal_value=config["cb_matrix_diag_value"])

    engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(cb_matrix), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
                       optimizer_fn=SGD, lr = config["lr"], weight_decay=5.0e-4)



In [278]:
# exp = mlflow.get_experiment_by_name("OrdinalLoss")
# run_ids = [r.run_id for r in mlflow.list_run_infos(exp.experiment_id)]


# for run_id in run_ids:


# #[pd.read_csv(f"mlruns/{exp.experiment_id}/{run_id}/metrics") for run_id in run_ids]

# pd.read_csv(
#     "mlruns/0/8ed2dbeacce148b7a109bf263f98dc3d/metrics/accuracy_test", 
#     sep = " ", 
#     names = ["ts", "score", "step"])
